In [2]:
import os
os.chdir('..')
!pwd

/Users/krishna/Documents/Research/Simultaneous Classifier/folktables-experiments/multidecomp


In [3]:
import pandas as pd
import numpy as np

In [4]:
from bilevel.utils import *

# Exploratory analyis of reconstructed Adult dataset

In [5]:
df_adult = pd.read_csv('./data_small/adult_reconstruction.csv')

In [6]:
df_adult.shape

(49531, 14)

In [6]:
df_adult.head()

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,sex,native-country,income,occupation
0,20,40,0,0,Private,Bachelors,13,Married-civ-spouse,Wife,White,Female,United-States,49100,Tech-support
1,40,21,0,0,Private,Some-college,10,Divorced,Own-child,White,Male,United-States,11500,Craft-repair
2,10,17,0,0,Private,11th,7,Never-married,Own-child,White,Male,United-States,2600,Other-service
3,50,51,0,0,Private,HS-grad,9,Married-civ-spouse,Husband,Asian-Pac-Islander,Male,Cambodia,38997,Sales
4,38,28,0,0,Private,Bachelors,13,Never-married,Not-in-family,White,Male,?,41400,Exec-managerial


In [7]:
df_adult.isnull().any(axis=1)

0        False
1        False
2        False
3        False
4        False
         ...  
49526    False
49527    False
49528    False
49529    False
49530    False
Length: 49531, dtype: bool

In [8]:
df_adult.isnull().sum()

hours-per-week    0
age               0
capital-gain      0
capital-loss      0
workclass         0
education         0
education-num     0
marital-status    0
relationship      0
race              0
sex               0
native-country    0
income            0
occupation        0
dtype: int64

In [8]:
df_adult.columns, len(df_adult.columns)

(Index(['hours-per-week', 'age', 'capital-gain', 'capital-loss', 'workclass',
        'education', 'education-num', 'marital-status', 'relationship', 'race',
        'sex', 'native-country', 'income', 'occupation'],
       dtype='object'),
 14)

In [9]:
numeric_feat = ['hours-per-week', 'age', 'capital-gain', 'capital-loss', 'education-num']
cat_feat = ['workclass', 'education', 'marital-status', 'relationship', 'race', 'sex', 'native-country', 'occupation']
print(len(numeric_feat), len(cat_feat))

5 8


In [10]:
# categoricals
from collections import Counter
for c in cat_feat:
    dic = Counter(df_adult[c])
    str = ""
    if '?' in dic:
        str = '?'
    print(c, str ,Counter(df_adult[c]))

# Takewaya workclass, native country, occupation have '?' category for unknown


workclass ? Counter({'Private': 34486, 'Self-emp-not-inc': 3871, 'Local-gov': 3157, '?': 2859, 'State-gov': 1990, 'Self-emp-inc': 1698, 'Federal-gov': 1439, 'Without-pay': 21, 'Never-worked': 10})
education  Counter({'HS-grad': 16086, 'Some-college': 11066, 'Bachelors': 8109, 'Masters': 2668, 'Assoc-voc': 2066, '11th': 1852, 'Assoc-acdm': 1608, '10th': 1405, '7th-8th': 962, 'Prof-school': 835, '9th': 761, '12th': 662, 'Doctorate': 595, '5th-6th': 525, '1st-4th': 248, 'Preschool': 83})
marital-status  Counter({'Married-civ-spouse': 22737, 'Never-married': 16398, 'Divorced': 6679, 'Separated': 1532, 'Widowed': 1519, 'Married-spouse-absent': 629, 'Married-AF-spouse': 37})
relationship  Counter({'Husband': 20065, 'Not-in-family': 12699, 'Own-child': 7764, 'Unmarried': 5151, 'Wife': 2339, 'Other-relative': 1513})
race  Counter({'White': 42441, 'Black': 4694, 'Asian-Pac-Islander': 1519, 'Amer-Indian-Eskimo': 471, 'Other': 406})
sex  Counter({'Male': 33174, 'Female': 16357})
native-country ? 

In [11]:
# numerics
df_adult[numeric_feat].describe()

,hours-per-week,age,capital-gain,capital-loss,education-num
count,49531.000000,49531.000000,49531.000000,49531.000000,49531.000000
mean,40.399184,38.554320,1064.819144,86.285114,10.069795
std,12.340824,13.701025,7401.671807,400.322887,2.565829
min,1.000000,17.000000,0.000000,0.000000,1.000000
25%,40.000000,28.000000,0.000000,0.000000,9.000000
50%,40.000000,37.000000,0.000000,0.000000,10.000000
75%,45.000000,48.000000,0.000000,0.000000,12.000000
max,99.000000,90.000000,99999.000000,4356.000000,16.000000


In [13]:
df_adult['income'].describe()

count    49531.000000
mean     33509.843129
std      27051.436074
min        107.000000
25%      12027.000000
50%      26013.000000
75%      48457.500000
max      99999.000000
Name: income, dtype: float64

In [23]:
# groups
race_groups = df_adult['race'].unique()
sex_groups = df_adult['sex'].unique()
occp_groups = df_adult['occupation'].unique()
race_groups, sex_groups, occp_groups, len(occp_groups)

(array(['White', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other',
        'Black'], dtype=object),
 array(['Female', 'Male'], dtype=object),
 array(['Tech-support', 'Craft-repair', 'Other-service', 'Sales',
        'Exec-managerial', 'Prof-specialty', '?', 'Handlers-cleaners',
        'Machine-op-inspct', 'Adm-clerical', 'Farming-fishing',
        'Transport-moving', 'Priv-house-serv', 'Protective-serv',
        'Armed-Forces'], dtype=object),
 15)

In [14]:
cat_feat

['workclass',
 'education',
 'marital-status',
 'relationship',
 'race',
 'sex',
 'native-country',
 'occupation']

In [10]:
numerics_all = numeric_feat + ['income']
print(numerics_all)

['hours-per-week', 'age', 'capital-gain', 'capital-loss', 'education-num', 'income']


In [13]:
df_adult_mm = numeric_scaler(df_adult, numerics_all)
df_adult_mm.head()

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,sex,native-country,income,occupation
0,0.193878,0.315068,0.0,0.0,Private,Bachelors,0.800000,Married-civ-spouse,Wife,White,Female,United-States,0.490460,Tech-support
1,0.397959,0.054795,0.0,0.0,Private,Some-college,0.600000,Divorced,Own-child,White,Male,United-States,0.114053,Craft-repair
2,0.091837,0.000000,0.0,0.0,Private,11th,0.400000,Never-married,Own-child,White,Male,United-States,0.024957,Other-service
3,0.500000,0.465753,0.0,0.0,Private,HS-grad,0.533333,Married-civ-spouse,Husband,Asian-Pac-Islander,Male,Cambodia,0.389320,Sales
4,0.377551,0.150685,0.0,0.0,Private,Bachelors,0.800000,Never-married,Not-in-family,White,Male,?,0.413376,Exec-managerial


In [17]:
df_adult_mm_le = ordinal_encoder(df_adult_mm, cat_feat)
df_adult_mm_le.head()

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,sex,native-country,income,occupation
0,0.193878,0.315068,0.0,0.0,4.0,9.0,0.800000,2.0,5.0,4.0,0.0,39.0,0.490460,13.0
1,0.397959,0.054795,0.0,0.0,4.0,15.0,0.600000,0.0,3.0,4.0,1.0,39.0,0.114053,3.0
2,0.091837,0.000000,0.0,0.0,4.0,1.0,0.400000,4.0,3.0,4.0,1.0,39.0,0.024957,8.0
3,0.500000,0.465753,0.0,0.0,4.0,11.0,0.533333,2.0,0.0,1.0,1.0,1.0,0.389320,12.0
4,0.377551,0.150685,0.0,0.0,4.0,9.0,0.800000,4.0,1.0,4.0,1.0,0.0,0.413376,4.0


In [15]:
df_adult_mm_oh = one_hot(df_adult_mm, cat_feat)
df_adult_mm_oh.drop(cat_feat, axis = 1, inplace=True) # drop the categorics that were used to one hot encode
df_adult_mm_oh = df_adult_mm_oh * 1.0 # make bool true, false into 1.0, 0.0
df_adult_mm_oh.head()

,hours-per-week,age,capital-gain,capital-loss,education-num,income,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving
0,0.193878,0.315068,0.0,0.0,0.800000,0.490460,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.397959,0.054795,0.0,0.0,0.600000,0.114053,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.091837,0.000000,0.0,0.0,0.400000,0.024957,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.500000,0.465753,0.0,0.0,0.533333,0.389320,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.377551,0.150685,0.0,0.0,0.800000,0.413376,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Benchmarking ML batch algorithms

In [19]:
from sklearn.model_selection import train_test_split
random_seed = 21
X_train, X_test, y_train, y_test = train_test_split(df_adult_mm_oh.drop('income', axis=1), 
                                                    df_adult_mm_oh['income'], test_size = 0.2,
                                                    shuffle=True, random_state= random_seed)

In [20]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import tree

In [21]:
lr = LinearRegression()
lr.fit(X_train, y_train)
print(lr.score(X_train, y_train), lr.score(X_test, y_test))

0.5839699444945836 -214714363240219.47


In [22]:
ridge = Ridge()
ridge.fit(X_train, y_train)
print(ridge.score(X_train, y_train), ridge.score(X_test, y_test))

0.5839681387328852 0.5835313399697666


In [24]:
df_le = pd.read_pickle('./data_small/dataframes/df_adultog_le.pkl')
random_seed = 21
X_train_le, X_test_le, y_train, y_test = train_test_split(df_le.drop('income', axis=1), 
                                                    df_le['income'], test_size = 0.2,
                                                    shuffle=True, random_state= random_seed)

In [25]:
X_train_le.shape, y_train.shape, X_test_le.shape, y_test.shape

((39624, 13), (39624,), (9907, 13), (9907,))

In [26]:
# gbr = GradientBoostingRegressor(max_depth = 10, n_estimators = 1000, learning_rate = 0.05)
gbr = GradientBoostingRegressor(max_depth = 4, n_estimators = 700, learning_rate = 0.1)
gbr.fit(X_train_le, y_train)
print(gbr.score(X_train_le, y_train), gbr.score(X_test_le, y_test))

0.7320034074105475 0.6985286344011372


In [ ]:
# # grid search for ``best'' gbr, in hyperopt/Adult_gridgbr.py
# param_grid = {
# "max_depth" = np.arange(3, 20),
# "learning_rate" = [0.01, 0.05, 0.1],
# "n_estimators" = np.arange(100, 1100, 200)
# }
# gbr_search = GradientBoostingRegressor()
# grid = GridSearchCV(estimator = gbr_search, 
#                     param_grid = param_grid, 
#                     scoring='r2',
#                     cv=5,
#                     n_jobs = 4, 
#                     verbose = 10)
# grid_result = grid_search.fit(X_train, y_train)


In [27]:
np.arange(100, 1100, 200)

array([100, 300, 500, 700, 900])

In [34]:
dtree_reg = tree.DecisionTreeRegressor(max_depth=11)
dtree_reg.fit(X_train_le, y_train)
print(dtree_reg.score(X_train_le, y_train), dtree_reg.score(X_test_le, y_test))

0.6887384232609746 0.6347359260510592


In [56]:
from sklearn.neighbors import KNeighborsRegressor
knn10 = KNeighborsRegressor(n_neighbors = 20) # not even as good as ridge
knn10.fit(X_train, y_train)
print(knn10.score(X_train, y_train), knn10.score(X_test, y_test))

0.6157087711671563 0.5791236932385222


# Data preparation sequence for ACS reconstructed

1. Sort by RACE, SEX or not
2. minmax scale numerics ['hours-per-week', 'age', 'capital-gain', 'capital-loss', 'education-num', 'income']

----
categorics ['workclass', 'education', 'marital-status', 'relationship', 'race', 'sex', 'native-country', 'occupation']

3. one hot encode, drop categories, save A_t, drop sensitive or not, shuffle seed
3. label encode, use the same above A_t, drop sensitive or not, use same above shuffle

## unshuffled original ordering in adult reconstructed, one hot and label encoded
- not sorted by sensitive groups(race, sex)
- just minmax scaled, numeric
- one hot encode, drop cat og

In [11]:
numeric_all = ['hours-per-week', 'age', 'capital-gain', 'capital-loss', 'education-num', 'income']
cat_feat = ['workclass', 'education', 'marital-status', 'relationship', 'race', 'sex', 'native-country', 'occupation']
print(len(numeric_all), len(cat_feat))

6 8


In [12]:
df_adult = pd.read_csv('./data_small/adult_reconstruction.csv')

In [13]:
df_adult_mm = numeric_scaler(df_adult, numeric_all)
df_adult_mm_oh = one_hot(df_adult_mm, cat_feat)
df_adult_mm_oh.drop(cat_feat, axis = 1, inplace=True) # drop the categorics that were used to one hot encode
df_adult_mm_oh = df_adult_mm_oh * 1.0 # make bool true, false into 1.0, 0.0
df_adult_mm_oh.head()

,hours-per-week,age,capital-gain,capital-loss,education-num,income,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving
0,0.193878,0.315068,0.0,0.0,0.800000,0.490460,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.397959,0.054795,0.0,0.0,0.600000,0.114053,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.091837,0.000000,0.0,0.0,0.400000,0.024957,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.500000,0.465753,0.0,0.0,0.533333,0.389320,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.377551,0.150685,0.0,0.0,0.800000,0.413376,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# batch ls vs ridge norm
from sklearn.linear_model import LinearRegression, Ridge
X_dat = df_adult_mm_oh.drop('income', axis=1)
y_dat = df_adult_mm_oh['income']

lr = LinearRegression()
lr.fit(X_dat, y_dat)
lr.score(X_dat,y_dat)


0.5840032172602747

In [15]:
ridge = Ridge()
ridge.fit(X_dat, y_dat)
ridge.score(X_dat,y_dat)

0.5840667219430855

In [18]:
lr.coef_

array([ 3.18116410e-01,  1.61345556e-01,  6.47526932e-01,  2.61356974e-01,
       -5.08594287e+11,  3.19015104e+11, -1.21112934e+11, -1.21112934e+11,
        3.19015104e+11, -1.21112934e+11, -1.21112934e+11, -1.21112934e+11,
       -1.21112934e+11, -1.21112934e+11, -2.74161717e+11, -2.40255431e+11,
       -2.06349146e+11, -4.09786860e+11, -3.75880574e+11, -3.41974289e+11,
       -3.08068003e+11, -7.07240025e+10, -1.04630288e+11, -3.68177168e+10,
        6.49011406e+10, -1.72442860e+11, -2.91143099e+09, -4.43693146e+11,
        3.09948548e+10, -1.38536574e+11,  1.93332790e+11,  1.93332790e+11,
        1.93332790e+11,  1.93332790e+11,  1.93332790e+11,  1.93332790e+11,
        1.93332790e+11, -1.24848247e+11, -1.24848247e+11, -1.24848247e+11,
       -1.24848247e+11, -1.24848247e+11, -1.24848247e+11,  3.13021233e+11,
        3.13021233e+11,  3.13021233e+11,  3.13021233e+11,  3.13021233e+11,
        4.31561125e+11,  4.31561125e+11, -4.90438075e+11, -4.90438075e+11,
       -4.90438075e+11, -

In [19]:
np.linalg.norm(ridge.coef_), np.linalg.norm(lr.coef_)

(0.9237039324090596, 3723921323703.2974)

In [12]:
race_groups = []
sex_groups = []
for col in df_adult_mm_oh.columns:
    if 'race' in col:
        race_groups.append(col)
    if 'sex' in col:
        sex_groups.append(col)
print(race_groups, sex_groups)

(['race_Amer-Indian-Eskimo',
  'race_Asian-Pac-Islander',
  'race_Black',
  'race_Other',
  'race_White'],
 ['sex_Female', 'sex_Male'])

## the sensitive groups

In [16]:
race_groups+sex_groups

['race_Amer-Indian-Eskimo',
 'race_Asian-Pac-Islander',
 'race_Black',
 'race_Other',
 'race_White',
 'sex_Female',
 'sex_Male']

In [25]:
def get_A_t_from_onehotdf(df_oh, groups, always_on = False) -> np.ndarray:
    A_tdf = df_oh[groups]
    if always_on:
        A_tdf['alwayson'] = 1
    return A_tdf.to_numpy()

In [18]:
A_tog = get_A_t_from_onehotdf(df_adult_mm_oh, race_groups+sex_groups, always_on = True)

/tmp/ipykernel_3191807/1068999074.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  A_tdf['alwayson'] = 1


In [21]:
np.save('./data_small/nparrays/A_t_adultog', A_tog)

In [24]:
df_adult_mm_oh.to_pickle('./data_small/dataframes/df_adultog_oh.pkl') # mm then oh (dropped categorics)

In [35]:
df_adult_mm_le = ordinal_encoder(df_adult_mm, cat_feat)
df_adult_mm_le.head()

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,sex,native-country,income,occupation
0,0.193878,0.315068,0.0,0.0,4.0,9.0,0.800000,2.0,5.0,4.0,0.0,39.0,0.490460,13.0
1,0.397959,0.054795,0.0,0.0,4.0,15.0,0.600000,0.0,3.0,4.0,1.0,39.0,0.114053,3.0
2,0.091837,0.000000,0.0,0.0,4.0,1.0,0.400000,4.0,3.0,4.0,1.0,39.0,0.024957,8.0
3,0.500000,0.465753,0.0,0.0,4.0,11.0,0.533333,2.0,0.0,1.0,1.0,1.0,0.389320,12.0
4,0.377551,0.150685,0.0,0.0,4.0,9.0,0.800000,4.0,1.0,4.0,1.0,0.0,0.413376,4.0


In [36]:
# np.all((A_tog[:,-3] == df_adult_mm_le['sex']) == False) 

True

In [37]:
df_adult_mm_le.to_pickle('./data_small/dataframes/df_adultog_le.pkl') # mm then label encoded

## sort by race, largest to smallest; mergesort

In [38]:
from collections import Counter
Counter(df_adult['sex']), Counter(df_adult['race'])

(Counter({'Male': 33174, 'Female': 16357}),
 Counter({'White': 42441,
          'Black': 4694,
          'Asian-Pac-Islander': 1519,
          'Amer-Indian-Eskimo': 471,
          'Other': 406}))

In [39]:
Counter(df_adult_mm['sex']), Counter(df_adult_mm['race'])

(Counter({'Male': 33174, 'Female': 16357}),
 Counter({'White': 42441,
          'Black': 4694,
          'Asian-Pac-Islander': 1519,
          'Amer-Indian-Eskimo': 471,
          'Other': 406}))

In [42]:
df_adult_mm_racels = df_adult_mm.sort_values(by='race', key=lambda x: x.map(x.value_counts()), ascending=False, inplace = False, kind = 'mergesort')

In [43]:
df_adult_mm_racels

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,sex,native-country,income,occupation
0,0.193878,0.315068,0.0,0.00000,Private,Bachelors,0.800000,Married-civ-spouse,Wife,White,Female,United-States,0.490460,Tech-support
1,0.397959,0.054795,0.0,0.00000,Private,Some-college,0.600000,Divorced,Own-child,White,Male,United-States,0.114053,Craft-repair
2,0.091837,0.000000,0.0,0.00000,Private,11th,0.400000,Never-married,Own-child,White,Male,United-States,0.024957,Other-service
4,0.377551,0.150685,0.0,0.00000,Private,Bachelors,0.800000,Never-married,Not-in-family,White,Male,?,0.413376,Exec-managerial
5,0.377551,0.123288,0.0,0.43067,Private,Bachelors,0.800000,Never-married,Not-in-family,White,Male,United-States,0.384585,Exec-managerial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48741,0.295918,0.164384,0.0,0.00000,Private,HS-grad,0.533333,Divorced,Not-in-family,Other,Female,Germany,0.241190,Prof-specialty
48927,0.397959,0.246575,0.0,0.00000,Local-gov,Some-college,0.600000,Married-civ-spouse,Husband,Other,Male,United-States,0.822318,Protective-serv
49115,0.704082,0.205479,0.0,0.00000,Private,Some-college,0.600000,Separated,Unmarried,Other,Female,United-States,0.200176,Other-service
49117,0.397959,0.109589,0.0,0.00000,Private,HS-grad,0.533333,Married-civ-spouse,Own-child,Other,Male,United-States,0.299253,Transport-moving


In [44]:
df_adult_mm_oh_racels = one_hot(df_adult_mm_racels, cat_feat)
df_adult_mm_oh_racels.drop(cat_feat, axis = 1, inplace=True) # drop the categorics that were used to one hot encode
df_adult_mm_oh_racels = df_adult_mm_oh_racels * 1.0 # make bool true, false into 1.0, 0.0
df_adult_mm_oh_racels.head()

,hours-per-week,age,capital-gain,capital-loss,education-num,income,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving
0,0.193878,0.315068,0.0,0.00000,0.8,0.490460,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.397959,0.054795,0.0,0.00000,0.6,0.114053,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.091837,0.000000,0.0,0.00000,0.4,0.024957,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.377551,0.150685,0.0,0.00000,0.8,0.413376,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.377551,0.123288,0.0,0.43067,0.8,0.384585,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
df_adult_mm_oh_racels.to_pickle('./data_small/dataframes/df_adult_racesorted_oh.pkl') # mm, sorted by race large-small, one hot 

In [46]:
race_groups+sex_groups

['race_Amer-Indian-Eskimo',
 'race_Asian-Pac-Islander',
 'race_Black',
 'race_Other',
 'race_White',
 'sex_Female',
 'sex_Male']

In [47]:
A_t_racels = get_A_t_from_onehotdf(df_adult_mm_oh_racels, race_groups + sex_groups, always_on = True)

/tmp/ipykernel_3191807/1068999074.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  A_tdf['alwayson'] = 1


In [48]:
np.save('./data_small/nparrays/A_t_racels', A_t_racels)

In [50]:
# np.all(df_adult_mm_oh_racels['race_White'] == A_t_racels[:, 4])

True

In [53]:
df_adult_mm_le_racels = ordinal_encoder(df_adult_mm_racels, cat_feat)
df_adult_mm_le_racels.head()

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,sex,native-country,income,occupation
0,0.193878,0.315068,0.0,0.00000,4.0,9.0,0.8,2.0,5.0,4.0,0.0,39.0,0.490460,13.0
1,0.397959,0.054795,0.0,0.00000,4.0,15.0,0.6,0.0,3.0,4.0,1.0,39.0,0.114053,3.0
2,0.091837,0.000000,0.0,0.00000,4.0,1.0,0.4,4.0,3.0,4.0,1.0,39.0,0.024957,8.0
4,0.377551,0.150685,0.0,0.00000,4.0,9.0,0.8,4.0,1.0,4.0,1.0,0.0,0.413376,4.0
5,0.377551,0.123288,0.0,0.43067,4.0,9.0,0.8,4.0,1.0,4.0,1.0,39.0,0.384585,4.0


In [54]:
df_adult_mm_le_racels.to_pickle('./data_small/dataframes/df_adult_racesorted_le.pkl')

## sorted by sex, largest to smallest

In [56]:
df_adult_mm_sexls = df_adult_mm.sort_values(by='sex', key=lambda x: x.map(x.value_counts()), ascending=False, 
                                             inplace = False, kind = 'mergesort')
df_adult_mm_sexls.head()

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,sex,native-country,income,occupation
1,0.397959,0.054795,0.0,0.00000,Private,Some-college,0.600000,Divorced,Own-child,White,Male,United-States,0.114053,Craft-repair
2,0.091837,0.000000,0.0,0.00000,Private,11th,0.400000,Never-married,Own-child,White,Male,United-States,0.024957,Other-service
3,0.500000,0.465753,0.0,0.00000,Private,HS-grad,0.533333,Married-civ-spouse,Husband,Asian-Pac-Islander,Male,Cambodia,0.389320,Sales
4,0.377551,0.150685,0.0,0.00000,Private,Bachelors,0.800000,Never-married,Not-in-family,White,Male,?,0.413376,Exec-managerial
5,0.377551,0.123288,0.0,0.43067,Private,Bachelors,0.800000,Never-married,Not-in-family,White,Male,United-States,0.384585,Exec-managerial


In [57]:
df_adult_mm_oh_sexls = one_hot(df_adult_mm_sexls, cat_feat)
df_adult_mm_oh_sexls.drop(cat_feat, axis = 1, inplace=True) # drop the categorics that were used to one hot encode
df_adult_mm_oh_sexls = df_adult_mm_oh_sexls * 1.0 # make bool true, false into 1.0, 0.0
df_adult_mm_oh_sexls.head()

,hours-per-week,age,capital-gain,capital-loss,education-num,income,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving
1,0.397959,0.054795,0.0,0.00000,0.600000,0.114053,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.091837,0.000000,0.0,0.00000,0.400000,0.024957,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.500000,0.465753,0.0,0.00000,0.533333,0.389320,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.377551,0.150685,0.0,0.00000,0.800000,0.413376,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.377551,0.123288,0.0,0.43067,0.800000,0.384585,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
df_adult_mm_oh_sexls.to_pickle('./data_small/dataframes/df_adult_sexsorted_oh.pkl')

In [60]:
sens_groups = race_groups + sex_groups

In [61]:
A_t_sexls = get_A_t_from_onehotdf(df_adult_mm_oh_sexls, sens_groups , always_on = True)

/tmp/ipykernel_3191807/1068999074.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  A_tdf['alwayson'] = 1


In [64]:
# A_t_sexls[:, -2] == df_adult_mm_oh_sexls['sex_Male']

1        True
2        True
3        True
4        True
5        True
         ... 
49516    True
49517    True
49518    True
49529    True
49530    True
Name: sex_Male, Length: 49531, dtype: bool

In [65]:
np.save('./data_small/nparrays/A_t_sexls', A_t_sexls)

In [66]:
A_t_sexls.shape

(49531, 8)

In [67]:
df_adult_mm_le_sexls = ordinal_encoder(df_adult_mm_sexls, cat_feat)
df_adult_mm_le_sexls.head()

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,sex,native-country,income,occupation
1,0.397959,0.054795,0.0,0.00000,4.0,15.0,0.600000,0.0,3.0,4.0,1.0,39.0,0.114053,3.0
2,0.091837,0.000000,0.0,0.00000,4.0,1.0,0.400000,4.0,3.0,4.0,1.0,39.0,0.024957,8.0
3,0.500000,0.465753,0.0,0.00000,4.0,11.0,0.533333,2.0,0.0,1.0,1.0,1.0,0.389320,12.0
4,0.377551,0.150685,0.0,0.00000,4.0,9.0,0.800000,4.0,1.0,4.0,1.0,0.0,0.413376,4.0
5,0.377551,0.123288,0.0,0.43067,4.0,9.0,0.800000,4.0,1.0,4.0,1.0,39.0,0.384585,4.0


In [68]:
df_adult_mm_le_sexls.to_pickle('./data_small/dataframes/df_adult_sexsorted_le.pkl')

### Occupation as groups

In [19]:
occp_groups

array(['Tech-support', 'Craft-repair', 'Other-service', 'Sales',
       'Exec-managerial', 'Prof-specialty', '?', 'Handlers-cleaners',
       'Machine-op-inspct', 'Adm-clerical', 'Farming-fishing',
       'Transport-moving', 'Priv-house-serv', 'Protective-serv',
       'Armed-Forces'], dtype=object)

In [20]:
numeric_all = ['hours-per-week', 'age', 'capital-gain', 'capital-loss', 'education-num', 'income']
cat_feat = ['workclass', 'education', 'marital-status', 'relationship', 'race', 'sex', 'native-country', 'occupation']
print(len(numeric_all), len(cat_feat))
df_adult = pd.read_csv('./data_small/adult_reconstruction.csv')

6 8


In [21]:
df_adult_mm = numeric_scaler(df_adult, numeric_all)
df_adult_mm_oh = one_hot(df_adult_mm, cat_feat)
df_adult_mm_oh.drop(cat_feat, axis = 1, inplace=True) # drop the categorics that were used to one hot encode
df_adult_mm_oh = df_adult_mm_oh * 1.0 # make bool true, false into 1.0, 0.0
df_adult_mm_oh.head()

,hours-per-week,age,capital-gain,capital-loss,education-num,income,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving
0,0.193878,0.315068,0.0,0.0,0.800000,0.490460,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.397959,0.054795,0.0,0.0,0.600000,0.114053,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.091837,0.000000,0.0,0.0,0.400000,0.024957,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.500000,0.465753,0.0,0.0,0.533333,0.389320,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.377551,0.150685,0.0,0.0,0.800000,0.413376,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
occ_groups_oh = []
for col in df_adult_mm_oh.columns:
    if 'occu' in col:
        occ_groups_oh.append(col)
print(occ_groups_oh, len(occ_groups_oh))

['occupation_?', 'occupation_Adm-clerical', 'occupation_Armed-Forces', 'occupation_Craft-repair', 'occupation_Exec-managerial', 'occupation_Farming-fishing', 'occupation_Handlers-cleaners', 'occupation_Machine-op-inspct', 'occupation_Other-service', 'occupation_Priv-house-serv', 'occupation_Prof-specialty', 'occupation_Protective-serv', 'occupation_Sales', 'occupation_Tech-support', 'occupation_Transport-moving'] 15


In [57]:
# A_t_occp = get_A_t_from_onehotdf(df_adult_mm_oh, occ_groups_oh, always_on = True)
A_t_occp = get_A_t_from_onehotdf(df_adult_mm_oh, occ_groups_oh, always_on = False)

In [60]:
A_t_occp.shape

(49531, 15)

In [31]:
# np.all(A_t_occp[:, :15] == df_adult_mm_oh[occ_groups_oh])
np.save('./data_small/nparrays/A_toccp', A_t_occp)

In [35]:
df_adult_mm_oh.to_pickle('./data_small/dataframes/df_adult_og_oh_occp.pkl')

### testing online performance on occp groups

In [41]:
import bilevel
from bilevel.Adahedge import *
from bilevel.build_all_models import *
from bilevel.OnlineLinearExpert import *
from bilevel.OnlineTreeExpert import *
import numpy as np
import pandas as pd

In [61]:
df_occp = pd.read_pickle('./data_small/dataframes/df_adult_og_oh_occp.pkl')
X_dat_og_oh = df_occp.drop('income', axis=1)
y_dat_og_oh = df_occp['income']
# A_t_occp = np.load('./data_small/nparrays/A_toccp.npy')

dirname = './models_adult/baseline/'
filename = 'ridge_occp'

expridge = OnlineLinearExpert(X_dat_og_oh, y_dat_og_oh, l2_pen = 1.0)
b_ridgebase = build_baseline_alwayson(dirname, filename, A_t_occp, expridge)

100%|██████████| 49531/49531 [00:30<00:00, 1608.89it/s]


In [62]:
dirname = './models_adult/Anh/'
filename = 'ridgemeta_occp'

experts = [OnlineLinearExpert(X_dat_og_oh, y_dat_og_oh, l2_pen = 1.0) for _ in range(A_t_occp.shape[1])]
b_Anh = build_Anh(dirname, filename, A_t_occp, experts)


100%|██████████| 49531/49531 [00:32<00:00, 1512.35it/s]


In [63]:
cumloss_groupwise_ada = b_Anh.Anh.cumloss_groupwise_ada
cumloss_base = b_ridgebase.expert.cumloss_groupwise
for g_ind, gname in enumerate(occ_groups_oh):
    base = cumloss_base[g_ind][-1]
    ada = cumloss_groupwise_ada[g_ind][-1]
    print(gname, base, ada , base >= ada) 

occupation_? 128.32610428021982 100.79875335332625 True
occupation_Adm-clerical 170.95383882019888 175.4571575219889 False
occupation_Armed-Forces 0.5475481872229374 1.4525682574671135 False
occupation_Craft-repair 241.88609302376375 254.1104830283236 False
occupation_Exec-managerial 449.93690025372 391.3662250420075 True
occupation_Farming-fishing 69.03776452805015 63.40533762028466 True
occupation_Handlers-cleaners 61.369998336587756 45.85498341549912 True
occupation_Machine-op-inspct 80.40866258872735 79.07199385429425 True
occupation_Other-service 136.29326513032262 94.48383683232069 True
occupation_Priv-house-serv 7.531286815521961 2.8098877307747663 True
occupation_Prof-specialty 459.1698447866027 388.10816984337737 True
occupation_Protective-serv 44.98667588881421 44.27594636283556 True
occupation_Sales 298.69001809925817 296.5239677571457 True
occupation_Tech-support 63.32653524964512 63.06972073582327 True
occupation_Transport-moving 89.73719277731873 90.74129431651424 False


In [ ]:
cumloss_groupwise_ada = b_Anh.Anh.cumloss_groupwise_ada # always active included helps!
cumloss_base = b_ridgebase.expert.cumloss_groupwise
for g_ind, gname in enumerate(occ_groups_oh):
    base = cumloss_base[g_ind][-1]
    ada = cumloss_groupwise_ada[g_ind][-1]
    print(gname, base, ada , base >= ada) 

occupation_? 128.32610428021982 101.12964194439633 True
occupation_Adm-clerical 170.95383882019888 174.56373464697404 False
occupation_Armed-Forces 0.5475481872229374 1.4525682574671135 False
occupation_Craft-repair 241.88609302376375 253.66854250279331 False
occupation_Exec-managerial 449.93690025372 392.44166482193543 True
occupation_Farming-fishing 69.03776452805015 63.098363116805324 True
occupation_Handlers-cleaners 61.369998336587756 45.93891544924698 True
occupation_Machine-op-inspct 80.40866258872735 78.40412185229627 True
occupation_Other-service 136.29326513032262 94.947785775834 True
occupation_Priv-house-serv 7.531286815521961 2.800883277425186 True
occupation_Prof-specialty 459.1698447866027 388.82693358243665 True
occupation_Protective-serv 44.98667588881421 43.96996639225248 True
occupation_Sales 298.69001809925817 295.70998891984755 True
occupation_Tech-support 63.32653524964512 62.657690579638434 True
occupation_Transport-moving 89.73719277731873 90.37650089210793 Fals

## Offline feature importances

- minmax scale, one hot encode and then find feature coeff of each
- ridge coeff or least squares p values

In [4]:
df_adult = pd.read_csv('./data_small/adult_reconstruction.csv')
print('number of cols adult', len(df_adult.columns))
numeric_all = ['hours-per-week', 'age', 'capital-gain', 'capital-loss', 'education-num', 'income']
cat_feat = ['workclass', 'education', 'marital-status', 'relationship', 'race', 'sex', 'native-country', 'occupation']
print(len(numeric_all), len(cat_feat))
df_adult_mm = numeric_scaler(df_adult, numeric_all)
df_adult_mm_oh = one_hot(df_adult_mm, cat_feat)
df_adult_mm_oh.drop(cat_feat, axis = 1, inplace=True) # drop the categorics that were used to one hot encode
df_adult_mm_oh = df_adult_mm_oh * 1.0 # make bool true, false into 1.0, 0.0

number of cols adult 14
6 8


In [5]:
df_adult_mm_oh.head()

,hours-per-week,age,capital-gain,capital-loss,education-num,income,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving
0,0.193878,0.315068,0.0,0.0,0.800000,0.490460,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.397959,0.054795,0.0,0.0,0.600000,0.114053,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.091837,0.000000,0.0,0.0,0.400000,0.024957,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.500000,0.465753,0.0,0.0,0.533333,0.389320,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.377551,0.150685,0.0,0.0,0.800000,0.413376,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
from sklearn.linear_model import LinearRegression, Ridge
X_dat = df_adult_mm_oh.drop('income', axis=1, inplace=False)
y_dat = df_adult_mm_oh['income']
ls = LinearRegression()
ls.fit(X_dat, y_dat)
ls.score(X_dat,y_dat)

0.5840032172602747

In [16]:
from sklearn.linear_model import LinearRegression, Ridge
X_dat = df_adult_mm_oh.drop('income', axis=1, inplace=False)
y_dat = df_adult_mm_oh['income']
ridge = Ridge()
ridge.fit(X_dat, y_dat)
ridge.score(X_dat,y_dat)

0.5840667219430855

In [50]:
from sklearn.linear_model import LinearRegression, Ridge
X_dat = df_adult_mm_oh.drop('income', axis=1, inplace=False)
y_dat = df_adult_mm_oh['income']
lr = LinearRegression()
lr.fit(X_dat, y_dat)
lr.score(X_dat,y_dat)

0.5840032172602747

In [51]:
lr.coef_

array([ 3.18116410e-01,  1.61345556e-01,  6.47526932e-01,  2.61356974e-01,
       -5.08594287e+11,  3.19015104e+11, -1.21112934e+11, -1.21112934e+11,
        3.19015104e+11, -1.21112934e+11, -1.21112934e+11, -1.21112934e+11,
       -1.21112934e+11, -1.21112934e+11, -2.74161717e+11, -2.40255431e+11,
       -2.06349146e+11, -4.09786860e+11, -3.75880574e+11, -3.41974289e+11,
       -3.08068003e+11, -7.07240025e+10, -1.04630288e+11, -3.68177168e+10,
        6.49011406e+10, -1.72442860e+11, -2.91143099e+09, -4.43693146e+11,
        3.09948548e+10, -1.38536574e+11,  1.93332790e+11,  1.93332790e+11,
        1.93332790e+11,  1.93332790e+11,  1.93332790e+11,  1.93332790e+11,
        1.93332790e+11, -1.24848247e+11, -1.24848247e+11, -1.24848247e+11,
       -1.24848247e+11, -1.24848247e+11, -1.24848247e+11,  3.13021233e+11,
        3.13021233e+11,  3.13021233e+11,  3.13021233e+11,  3.13021233e+11,
        4.31561125e+11,  4.31561125e+11, -4.90438075e+11, -4.90438075e+11,
       -4.90438075e+11, -

In [20]:
len(X_dat.columns), len(weights)

(107, 107)

In [38]:
weights

array([ 0.31916381,  0.16279071,  0.64556186,  0.26670961,  0.19557988,
       -0.02137973,  0.07141286,  0.00599138, -0.0094531 ,  0.01665241,
        0.05232974, -0.03803343, -0.0132648 , -0.06425533, -0.0326251 ,
       -0.02975413, -0.0289089 , -0.0335685 , -0.03379394, -0.05753852,
       -0.04304816, -0.01650891, -0.00887151,  0.03024889,  0.10164917,
       -0.02670332,  0.07467112, -0.00089757,  0.11966436, -0.01401497,
       -0.03185226,  0.07906486,  0.09914405, -0.02428762, -0.05094258,
       -0.02798277, -0.04314368,  0.026959  , -0.02146778, -0.03801699,
       -0.05241526, -0.02117601,  0.10611704, -0.02627936,  0.01319843,
       -0.00528588,  0.00949528,  0.00887153, -0.02043203,  0.02043203,
        0.00194121,  0.06462967,  0.0500984 , -0.0474109 , -0.05471827,
        0.01190914, -0.02460849, -0.02159106,  0.01834858,  0.04609866,
        0.02283327,  0.04136037, -0.00646955,  0.02733722,  0.01556246,
       -0.08256778, -0.0210501 , -0.0047336 , -0.04395377, -0.03

In [41]:
weights = ridge.coef_

ridge_weights = pd.DataFrame(np.hstack((X_dat.columns.to_numpy().reshape(-1,1), weights.reshape(-1,1))), columns = ['feat', 'coef'])
ridge_weights['abs_coef'] = abs(ridge_weights['coef'])

In [46]:
print(ridge_weights.sort_values(by = 'abs_coef', ascending = False).to_string())

                                          feat      coef  abs_coef
2                                 capital-gain  0.645562  0.645562
0                               hours-per-week  0.319164  0.319164
3                                 capital-loss   0.26671   0.26671
4                                education-num   0.19558   0.19558
1                                          age  0.162791  0.162791
28                       education_Prof-school  0.119664  0.119664
42                           relationship_Wife  0.106117  0.106117
24                         education_Doctorate  0.101649  0.101649
32           marital-status_Married-civ-spouse  0.099144  0.099144
96                  occupation_Exec-managerial  0.095926  0.095926
71                      native-country_Ireland   0.08546   0.08546
87                     native-country_Thailand -0.082631  0.082631
65           native-country_Holand-Netherlands -0.082568  0.082568
31            marital-status_Married-AF-spouse  0.079065  0.07

In [11]:
# weights = ls.coef_
# weights

In [47]:
import statsmodels.api as sm
import numpy as np

model = sm.OLS(y_dat, X_dat)
results = model.fit()
# results.summary()


In [48]:
# print(results.summary())
# print(results.pvalues.sort_values(ascending=True))

                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       0.584
Model:                            OLS   Adj. R-squared:                  0.583
Method:                 Least Squares   F-statistic:                     707.8
Date:                Mon, 11 Sep 2023   Prob (F-statistic):               0.00
Time:                        11:15:09   Log-Likelihood:                 16139.
No. Observations:               49531   AIC:                        -3.208e+04
Df Residuals:                   49432   BIC:                        -3.121e+04
Df Model:                          98                                         
Covariance Type:            nonrobust                                         
                                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------

In [49]:
print(results.pvalues.sort_values(ascending=True).to_string())

hours-per-week                                0.000000e+00
capital-gain                                  0.000000e+00
capital-loss                                 2.913679e-208
age                                          4.587961e-156
occupation_Armed-Forces                       6.335416e-02
occupation_Farming-fishing                    6.335416e-02
occupation_Priv-house-serv                    6.335416e-02
occupation_Other-service                      6.335416e-02
occupation_Handlers-cleaners                  6.335416e-02
occupation_Machine-op-inspct                  6.335416e-02
occupation_Transport-moving                   6.335416e-02
occupation_Adm-clerical                       6.335416e-02
occupation_Craft-repair                       6.335416e-02
occupation_Sales                              6.335416e-02
occupation_Protective-serv                    6.335416e-02
occupation_Prof-specialty                     6.335416e-02
occupation_Tech-support                       6.335416e-

In [ ]:
# education, occupation, hours worked per week, capital gain, 